### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [275]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## School District Summary

In [276]:
# District Summary

#Calculate the total number of schools
school_count = school_data['school_name'].count()

#Calculate the total number of students
student_count = student_data['Student ID'].count()

# Calculate the total budget
district_budget = school_data['budget'].sum()

# Calculate the average math score 
avg_math = student_data['math_score'].mean()

# Calculate the average reading score
avg_reading = student_data['reading_score'].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
pct_passing_math = (student_data.math_score >= 70).mean()

# Calculate the percentage of students with a passing reading score (70 or greater)
pct_passing_reading = (student_data.reading_score >= 70).mean()

# Calculate the percentage of students who passed math **and** reading (% Overall Passing)
pct_passing_mr = ((student_data.reading_score >= 70) & (student_data.math_score >= 70)).mean()

# Create a dataframe to hold the above results
district_summary_df = pd.DataFrame({
    'Total Schools': [school_count], 
    'Total Students': [student_count],
    'Total Budget': [district_budget],
    'Average Math Score': [avg_math],
    'Average Reading Score': [avg_reading],
    '% Passing Math': [pct_passing_math],
    '% Passing Reading': [pct_passing_reading],
    '% Passing Math and Reading': [pct_passing_mr]})

# Optional: give the displayed data cleaner formatting

format_dict = {'Total Budget':'${0:,.2f}', 'Total Students': '{:,}', 'Average Math Score': '{:.2f}', 'Average Reading Score': '{:.2f}', '% Passing Math': '{:.2%}', '% Passing Reading': '{:.2%}', '% Passing Math and Reading': '{:.2%}'}

district_summary_df = district_summary_df.style.format(format_dict).hide_index()


In [277]:
district_summary_df

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Math and Reading
15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

In [278]:
#update school df
#WARNING - THIS ONLY RUNS ONCE - IT WILL FAIL ONCE THE COLUMN HAS BEEN DROPPED - ALTERNATE WOULD BE TO CREATE A NEW DF W/O THIS COLUMN AND WORK AGAINST THAT GOING FORWARD - I CHOSE THIS METHOD
school_data = school_data.drop(columns = ['School ID'])
# Set the index
school_data = school_data.set_index('school_name')


In [279]:
#update student df
student_data['passing_both'] = ((student_data['math_score'] >= 70) & (student_data['reading_score'] >= 70))

#get school averages
stdt_grp = student_data.groupby(['school_name']) 

s_mean_math_score = stdt_grp['math_score'].mean()
s_mean_reading_score = stdt_grp['reading_score'].mean()

s_passing_math = stdt_grp['math_score'].apply(lambda x: (x >= 70).mean())
s_passing_reading = stdt_grp['reading_score'].apply(lambda x: (x >= 70).mean())

#% passing both - I couldn't figure out how to do with in groupby w lambda - would love feedback
s_passing_both = stdt_grp['passing_both'].mean()

#merge the data
merged_school_data = school_data.merge(s_mean_math_score,on='school_name').merge(s_mean_reading_score,on='school_name').merge(s_passing_math,on='school_name').merge(s_passing_reading,on='school_name').merge(s_passing_both,on='school_name')


#rename the columns
renamed_school_data = merged_school_data.rename(columns={'school_name': "School Name", 'type': 'School Type', 'size': 'Total Students', 'budget': 'Total School Budget', 'math_score_x': 'Average Math Score', 'reading_score_x': 'Average Reading Score', 'math_score_y': '% Passing Math', 'reading_score_y': '% Passing Reading', 'math_score_x': 'Average Math Score', 'passing_both': '% Overall Passing'})


#formatting

format_dict = {'Total School Budget':'${0:,.2f}', 'Total Students': '{:,}', 'Average Math Score': '{:.2f}', 'Average Reading Score': '{:.2f}', '% Passing Math': '{:.2%}', '% Passing Reading': '{:.2%}', '% Overall Passing': '{:.2%}'}

sorted_school_data = renamed_school_data.sort_index()
all_school_summary = sorted_school_data.style.format(format_dict)



In [280]:
all_school_summary

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",77.07,80.97,66.06%,81.22%,53.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [281]:
top_five = renamed_school_data.sort_values(by='% Overall Passing', ascending=False).head(5)
top_five_summary = top_five.style.format(format_dict)
top_five_summary

#top_five = renamed_school_data.style.format(format_dict).head(5)

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [282]:
bottom_five = renamed_school_data.sort_values(by='% Overall Passing', ascending=True).head(5)
bottom_five_summary = bottom_five.style.format(format_dict)
bottom_five_summary

,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [283]:
#I did the merge method above, so I am trying something different here.

#create a pivot table
import numpy as np
math_table = pd.pivot_table(school_data_complete, values='math_score', index=['school_name'], columns=['grade'], aggfunc=np.mean)

#reorder the columns
column_order = ['9th', '10th', '11th', '12th']
math_table1 = math_table.reindex(column_order, axis=1)

math_table1


grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [284]:
#create a pivot table
import numpy as np
reading_table = pd.pivot_table(school_data_complete, values='reading_score', index=['school_name'], columns=['grade'], aggfunc=np.mean)

#reorder the columns
column_order = ['9th', '10th', '11th', '12th']
reading_table1 = reading_table.reindex(column_order, axis=1)

reading_table1

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [301]:
#figure out the per student spending - I think the question misused the word 'average' related to school spending so I am ignoring it.  
renamed_school_data['spending_per_student'] = renamed_school_data['Total School Budget']/renamed_school_data['Total Students']

school_df = pd.DataFrame(renamed_school_data[['spending_per_student', 'Average Math Score', 'Average Math Score','% Passing Math', '% Passing Reading', '% Overall Passing']])

In [305]:
#based on the range, bins will be roughly 575 - 675 - but because the example uses oddly shaped bins, I do that as well instead of even ranges
bins = [0, 585, 630, 645, 675]
bin_names = ['<$584', '$585 - $629', '$630 - $644', '$645 - $675']

school_df['Spending Ranges (Per Student)'] = pd.cut(school_df['spending_per_student'], bins, labels=bin_names, include_lowest=True)

#Drop unwanted column.
school_df = school_df.drop(columns=['spending_per_student'])


In [306]:
#Return the final table 
spending_vs_scores = school_df.groupby(['Spending Ranges (Per Student)'])
spending_vs_scores.mean()


,Average Math Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.455399,83.455399,0.934601,0.966109,0.903695
$585 - $629,81.899826,81.899826,0.871335,0.927182,0.814186
$630 - $644,78.518855,78.518855,0.734842,0.843918,0.628577
$645 - $675,76.997210,76.997210,0.661648,0.811340,0.535269


## Scores by School Size

* Perform the same operations as above, based on school size.

In [312]:
#place data in df
size_df = pd.DataFrame(renamed_school_data[['Total Students', 'Average Math Score', 'Average Math Score','% Passing Math', '% Passing Reading', '% Overall Passing']])

In [315]:
#create bins and assign labels
bins = [0, 1000, 2000, 5000]
bin_names = ['Small <1000', 'Medium 1000 - 1999', 'Large 2000 - 5000']

size_df['School Size'] = pd.cut(size_df['Total Students'], bins, labels=bin_names, include_lowest=True)

#Drop unwanted column.
size_df = size_df.drop(columns=['Total Students'])

In [316]:
#Return the final table 
schl_size_vs_scores = size_df.groupby(['School Size'])
schl_size_vs_scores.mean()

,Average Math Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small <1000,83.821598,83.821598,0.935502,0.960994,0.898839
Medium 1000 - 1999,83.374684,83.374684,0.935997,0.967907,0.906215
Large 2000 - 5000,77.746417,77.746417,0.699634,0.827666,0.582860


## Scores by School Type

* Perform the same operations as above, based on school type

In [318]:
s_type_df = pd.DataFrame(renamed_school_data[['School Type', 'Average Math Score', 'Average Math Score','% Passing Math', '% Passing Reading', '% Overall Passing']])

In [320]:
type_vs_scores = s_type_df.groupby(['School Type'])
type_vs_scores.mean()

,Average Math Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.473852,0.936208,0.965865,0.904322
District,76.956733,76.956733,0.665485,0.807991,0.536722


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
